In [9]:
pip install spacy[transformers]

Note: you may need to restart the kernel to use updated packages.


In [10]:
import json
 
with open('/kaggle/input/d/lalithmovva/legal-ner/NER_TRAIN_PREAMBLE.json', 'r') as f:
    data = json.load(f)

In [11]:
training_data = {'classes' : [], 'annotations' : []}
for example in data:
  
  if len(example['annotations']) >= 1 and len(example['annotations'][0]['result']) >= 1:
    temp_dict = {}
    temp_dict['text'] = example['data']['text']
    temp_dict['entities'] = []
    start = example['annotations'][0]['result'][0]['value']['start']
    end = example['annotations'][0]['result'][0]['value']['end']
    label = example['annotations'][0]['result'][0]['value']['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)
        
  
print(training_data['annotations'][0])

{'text': "In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n", 'entities': [(7, 40, 'COURT')]}


In [12]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [13]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 1558/1558 [00:02<00:00, 519.96it/s]


In [14]:
!python -m spacy init fill-config /kaggle/input/d/lalithmovva/legal-ner/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-11-30 19:31:19,259] [INFO] Set up nlp object from config
[2022-11-30 19:31:19,270] [INFO] Pipeline: ['transformer', 'ner']
[2022-11-30 19:31:19,276] [INFO] Created vocabulary
[2022-11-30 19:31:19,277] [INFO] Finished initializing nlp object
Downloading: 100%|███████████████████████████| 29.0/29.0 [00:00<00:00, 26.9kB/s]
Downloading: 100%|██████████████████████████████| 411/411 [00:00<00:00, 412kB/s]
Downloading: 100%|███████████████████████████| 208k/208k [00:00<00:00, 1.14MB/s]
Downloading: 100%|███████████████████████████| 426k/426k [00:00<00:00, 1.74MB/s]
Downloading: 100%|███████████████████████████| 251M/251M [00:12<00:00, 21.8MB/s]
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_tr

In [22]:
import json
 
with open('/kaggle/input/legal-ner1/NER_DEV_PREAMBLE.json', 'r') as f:
    data1 = json.load(f)

In [35]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner(data1[11]['data']['text'])
print(data1[0]['annotations'][0]['result'][0]['value']['labels'])
colors = {"COURT": "#F67DE3", "JUDGE": "#7DF6D9", "PETITIONER":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

['COURT']
